# Clip Multi-source Lunar Data Using Shapefiles

*1. This notebook demonstrates how to clip multiple raster datasets (Optical image data, DEM data, spectral data, etc.)*  
*2. for each lunar crater using the previously generated shapefiles.*  
*3. The clipping process uses ArcPy.*

## Environment Information

- Python: 2.7(ArcGIS 10.8 environment)
- Key libraries:
    - arcpy
    - numpy 1.9.3
    - pandas 0.18.1
    - scipy 0.17.0
    - matplotlib 1.5.2
    - requests 2.11.1
    - python-dateutil 2.5.3
    - pytz 2016.6.1
    - six 1.10.0

## Import necessary libraries

In [2]:
import arcpy
import os,fnmatch
from arcpy import sa
from arcpy import env
import re
import numpy as np
import pandas as pd
from arcpy.sa import ExtractByMask, SetNull

## Define Function

In [3]:
# For CE-2 DEM data:
# Clip multiple rasters using their corresponding shapefiles, and set all pixel values ≤ -99999 to NoData.
def clip_multiple_data(other_rasterfile, frames_folderPath, mosaic_path):
    arcpy.CheckOutExtension("Spatial")  # 确保 Spatial Analyst 可用
    arcpy.env.overwriteOutput = True

    # obtain the ratser's projection
    prj = arcpy.Describe(other_rasterfile).spatialReference

    # Traverse all shapefiles
    shps = [os.path.join(dirpath, f)
            for dirpath, dirname, files in os.walk(frames_folderPath)
            for f in fnmatch.filter(files, '*.shp')]

    for shp in shps:
        # Get the shapefile projection
        Output_Coordinate_System = arcpy.Describe(shp).spatialReference
        tempEnvironment0 = arcpy.env.outputCoordinateSystem
        arcpy.env.outputCoordinateSystem = Output_Coordinate_System

        frame_name = os.path.basename(shp)[:-4]
        out_raster = mosaic_path

        # Step 1: Clip the raster using the shapefile
        clipped_raster = ExtractByMask(other_rasterfile, shp)

        # Step 2: Set all pixel values ≤ -99999 to NoData
        cleaned_raster = SetNull(clipped_raster, clipped_raster, "VALUE <= -99999")

        # Save the clipped and cleaned raster
        cleaned_raster.save(out_raster)

        # Restore the original output coordinate system
        arcpy.env.outputCoordinateSystem = tempEnvironment0

    arcpy.CheckInExtension("Spatial")

In [4]:
# Check whether the target shapefile (target_shp) has spatial overlaps 
# with each feature in the shapefiles corresponding to the TIFF files.
def get_overlapping_tif_indices(frames_folderPath, tif_shp_path, tif_path, wac_spatial_ref):
    overlapping_indices = []
    shps = [os.path.join(dirpath, f) for dirpath, dirname, files in os.walk(frames_folderPath) for f in
            fnmatch.filter(files, '*.shp')]
    shp = shps[0]
    # Get the geometry of the single feature in the target shapefile
    with arcpy.da.SearchCursor(shp, ["SHAPE@"]) as cursor:
        target_geometry = None
        for row in cursor:
            target_geometry = row[0]
            break

    target_spatial_ref = target_geometry.spatialReference

    # Use SearchCursor to iterate through all polygon features in the shapefile corresponding to the TIFF files
    with arcpy.da.SearchCursor(tif_shp_path, ["SHAPE@","FID","CE2_NAME"]) as cursor:
        for row in cursor:
            tif_geometry = row[0]
            tif_fid = row[1]
            dom_name = row[2].encode('utf-8') 
            dom_path = os.path.join(tif_path, dom_name)

            # Ensure both shapefiles use the same spatial reference
            target_spatial_ref = target_geometry.spatialReference
            tif_spatial_ref = tif_geometry.spatialReference

            target_geometry = target_geometry.projectAs(wac_spatial_ref)
            tif_geometry = tif_geometry.projectAs(wac_spatial_ref)

            # Check if the target feature overlaps or lies within the current polygon feature
            if target_geometry.overlaps(tif_geometry) or target_geometry.within(tif_geometry):
                overlapping_indices.append((tif_fid, dom_name, dom_path))

    return overlapping_indices

In [5]:
# Extract the latitude information from the name of the shapefile
def get_file_name_lat(frames_folderPath):
    shps = [os.path.join(dirpath, f) for dirpath, dirname, files in os.walk(frames_folderPath) for f in
            fnmatch.filter(files, '*.shp')]
    for shp in shps:
        filename = os.path.basename(shp)[:-4]
        # Extract latitude and longitude
        matches = list(re.finditer(r'[a-zA-Z]', filename))
        # Extract the positions of the matched letters
        position_of_letters = [match.start() for match in matches]
        # Extract the matched latitude longitude letters
        index_of_lon = position_of_letters[0]
        letters_lon = filename[index_of_lon]
        prefix_before_lon = filename[:index_of_lon]
        if letters_lon == 'E':
            lon_str = prefix_before_lon.replace('_', '.')
            lon = float(lon_str)
        elif letters_lon == 'W':
            lon_str = prefix_before_lon.replace('_', '.')
            lon = 360 - float(lon_str)

        index_of_lat = position_of_letters[1]
        letters_lat = filename[index_of_lat]
        prefix_before_lat = filename[index_of_lon + 1:index_of_lat]
        if letters_lat == 'N':
            lat_str = prefix_before_lat.replace('_', '.')
            lat = float(lat_str)
        elif letters_lat == 'S':
            lat_str = prefix_before_lat.replace('_', '.')
            lat = float(lat_str) * -1

    return filename,lat

In [6]:
# Check whether the shapefile is completely within the extent of the raster
def is_shapefile_within_raster(shapefile_folder, raster, test_path):
    # Obtain crater's shapefile information 
    u_shp_path = unicode(shapefile_folder, "utf8")
    shpfiles = os.listdir(u_shp_path)
    shps = [filename for filename in shpfiles if filename.lower().endswith('.shp')]
    shp_path = os.path.join(frames_folderPath, str(shps[0]))
    shp_path2 = os.path.normpath(shp_path)

    shapefile_desc = arcpy.Describe(shp_path2)
    shapefile_desc = arcpy.Describe(shp_path2)
    shapefile_extent = shapefile_desc.extent

    # Obtain crater's raster information
    raster_desc = arcpy.Describe(raster)
    raster_extent = raster_desc.extent
    raster_spatial_ref = raster_desc.spatialReference

    # Reproject the shapefile to ensure that the raster and shapefile have the same coordinate system
    projected_shp_path = test_path +"/" + str(i) + "_" + "projected_shp.shp"
    projected_shp_path = os.path.normpath(projected_shp_path)
    arcpy.Project_management(shp_path2,projected_shp_path,raster_spatial_ref)
    # Check whether the bounding box of the shapefile is completely within the bounding box of the raster
    with arcpy.da.SearchCursor(projected_shp_path, ["SHAPE@"]) as cursor:
        for row in cursor:
            shp_geom = row[0]
    if raster_extent.contains(shp_geom):
        arcpy.Delete_management(projected_shp_path)
        return 1
    else:
        arcpy.Delete_management(projected_shp_path)
        return 0

In [7]:
# Clip the global lunar raster
def clip_other_data(other_rasterfile, frames_folderPath, result_clipRasterPath):
    prj = arcpy.Describe(other_rasterfile).spatialReference
    shps = [os.path.join(dirpath, f) for dirpath, dirname, files in os.walk(frames_folderPath) for f in
            fnmatch.filter(files, '*.shp')]

    for shp in shps:
        Output_Coordinate_System = arcpy.Describe(shp).spatialReference
        tempEnvironment0 = arcpy.env.outputCoordinateSystem
        arcpy.env.outputCoordinateSystem = Output_Coordinate_System

        frame_name = os.path.basename(shp)[:-4]
        out_raster = result_clipRasterPath + '\\' + frame_name + '.tif'

        arcpy.gp.ExtractByMask_sa(other_rasterfile, shp, out_raster)
        arcpy.env.outputCoordinateSystem = tempEnvironment0

## Main Execution

### Define the path to the raster files, which were downloaded and processed in 02_Data_prepare (Python27)

In [11]:
data = pd.read_csv(r'./test/craters_read_7317_test.csv'.decode('utf-8'))
diameters = data.iloc[:, 2]
FID = data.iloc[:, 0]

# Define the path to the raster files, which were downloaded and processed in 02_Data_prepare (Python27)
test_path = "your output directory"
wacfile = r"your path of Lunar_LRO_LROC-WAC_Mosaic_global_100m_June2013.tif"  # your wac file's path
wacfile = unicode(wacfile, 'utf-8')
nacfile = r"./test/NAC/M1154129961LE.tif"# your nac file's path

ce2_folder = r"path to your CE-2 DEM data directory"  # replace your floder path of ce-2 dem
ce2_shp = r"./test/CE2DEM_shp/CE2_DEM_Boundaries.shp"
north_dem = r"your path of LDEM_60N_30M_merge.tif" # replace your lro north_dem's path
south_dem = r"your path of LDEM_60S_30M_merge.tif"# replace your lro south_dem's path
olivine = r"your path of olivine_90N90S.tif"# replace your olivine's path
plagioclase = r"your path of plagioclase_90N90S.tif"  # replace your plagioclase's path
clementine_Fe = r"your path of FeO.tif" # replace your Clementine UVVIS derived iron abundance (wt% FeO) map's path
clementine_Ti = r"your path of TiO2.tif"# replace your Clementine UVVIS derived titanium abundance (wt% TiO₂) map's path
grav_folder = r"your path of gggrx_1200a_boug_l900_projected.tif"# replace your Gravity Bouguer Disturbance map's path
He3_folder = r"your path of He3.tif" # replace your Clementine UVVIS derived ³He abundance maps' path
ling_Fe_folder = r"your path of IIM_global_161226_4x4_FeO_Ling14_2fit_LP_filter_GEO_new.tif"# replace your IIM derived iron abundance (wt% FeO) maps' path
ling_Ti_folder = r"your path of IIM_global_161226_4x4_TiO2_Ling16_2fit_LP_filter_GEO_new.tif"# replace your IIM derived titanium abundance (wt% TiO₂) maps' path

# replace your Moon Clementine UVVIS 5 Band Warped Image Mosaic 200m of the 415/750/900/950/1000 nm wavelength's path
spect1_folder = r"your path of UVVIS_Warp_Mosaic_5Bands_200m_cub_Band_1.tif"
spect2_folder = r"your path of UVVIS_Warp_Mosaic_5Bands_200m_cub_Band_2.tif"
spect3_folder = r"your path of UVVIS_Warp_Mosaic_5Bands_200m_cub_Band_3.tif"
spect4_folder = r"your path of UVVIS_Warp_Mosaic_5Bands_200m_cub_Band_4.tif"
spect5_folder = r"your path of UVVIS_Warp_Mosaic_5Bands_200m_cub_Band_5.tif"

# Set the hillshade local variables
inRaster = "elevation"
azimuth = 318
altitude = 45
modelShadows = "NO_SHADOWS"
zFactor = 1

In [9]:
data

,FID,filename,Diameter [km]
0,1,54_74E5_56N,10.40
1,2,17_28E4_55N,10.40
2,3,29_27E18_66N,10.55
3,4,71_49E3_87S,10.50
4,5,24_12W33_75N,10.66
5,6,81_37E1_32S,10.73
6,7,45_93E6_19N,10.91
7,8,28_62E86_99N,10.94
8,9,31_16W19_98N,10.98
9,225,17_92E21_73N,15.56


### Clip and create multi-source RS datasets for each crater sample

In [ ]:
prj = arcpy.Describe(wacfile).spatialReference
for num in range(9,len(FID)-1): 
    i = FID[num]
    dia_i = data.loc[FID == i, 'Diameter [km]'].values #the diameter of the i-th crater
    folderPath = r"G:/test/Crater_files_test/" + str(i) + "/" # Define the parent directory path of the shapefile (generate from step 1)  
    frames_folderPath = os.path.join(folderPath, "shp")
    
    """clip optical image"""
    result_clipRasterPath_tif = os.path.join(folderPath, "tif")
    os.makedirs(result_clipRasterPath_tif)
    if dia_i<1:
        print(i,'clip nac')
        clip_other_data(nacfile, frames_folderPath, result_clipRasterPath_tif)
    else:
        print(i,'clip wac')
        clip_other_data(wacfile, frames_folderPath, result_clipRasterPath_tif)
    
    """clip dem"""
    frame_name,lat = get_file_name_lat(frames_folderPath) #
    shp_path = frames_folderPath +'/' + frame_name + ".shp"
    result_clipRasterPath_dem = os.path.join(folderPath, "dem")
    os.makedirs(result_clipRasterPath_dem)
    if lat < -60:
        print(i)
        check_contain = is_shapefile_within_raster(frames_folderPath, south_dem,test_path)
        if check_contain == 1:
            # Clip the polar lro DEM if the shapefile is fully contained within the raster.
            clip_other_data(south_dem, frames_folderPath, result_clipRasterPath_dem)
        else:
            # If the shapefile is not fully contained within the polar LRO DEM, use the CE-2 DEM.
            print(str(i) + " is above -60°, but not fully contained within the polar DEM")

    elif lat > 60:
        print(i)
        check_contain = is_shapefile_within_raster(frames_folderPath, north_dem,test_path)
        if check_contain == 1:
            # Clip the polar lro DEM if the shapefile is fully contained within the raster.
            clip_other_data(north_dem, frames_folderPath, result_clipRasterPath_dem)

        else:
            # If the shapefile is not fully contained within the polar LRO DEM, use the CE-2 DEM.
            print(str(i) + " is above 60°, but not fully contained within the polar DEM")
    else:
        overlapping_indices = get_overlapping_tif_indices(frames_folderPath, ce2_shp, ce2_folder, prj)
        if len(overlapping_indices)==1:
            clip_other_data(overlapping_indices[0][2], frames_folderPath, result_clipRasterPath_dem)
        else:
            clipped_rasters = []
            overlapping_indices_fids = [data2[0] for data2 in overlapping_indices]
            for tif_fid, dom_name, dom_path in overlapping_indices:
                mosaic_path = os.path.join(result_clipRasterPath_dem, "clipped_{}.tif".format(tif_fid))
                clip_multiple_data(dom_path, frames_folderPath, mosaic_path)
                clipped_rasters.append(mosaic_path)
            print("CE-2 DEM tile incomplete; clip and mosaic required")

            merged_name = frame_name+ '.tif'
            arcpy.management.MosaicToNewRaster(clipped_rasters,
                                               result_clipRasterPath_dem,
                                               merged_name,
                                               pixel_type="32_BIT_FLOAT",
                                               number_of_bands=1,
                                               mosaic_method="LAST"
                                               )
            # Delete all intermediate clipped files
            for clipped_raster in clipped_rasters:
                arcpy.Delete_management(clipped_raster)
                
    """make hillshade"""
    normalized_path = os.path.normpath(result_clipRasterPath_dem + '\\' + str(frame_name)+'.tif')
    result_clipRasterPath_hillade = os.path.join(folderPath, 'hillshade')
    os.makedirs(result_clipRasterPath_hillade)
    output_file = os.path.join(result_clipRasterPath_hillade, str(frame_name)+'.tif')
    # Execute HillShade
    arcpy.gp.HillShade_sa(normalized_path, output_file, "315", "45", "NO_SHADOWS", "1")
                
    """clip Gravity Data"""
    result_clipRasterPath_grav = os.path.join(folderPath, "grav")
    os.makedirs(result_clipRasterPath_grav)
    clip_other_data(grav_folder, frames_folderPath, result_clipRasterPath_grav)
    
    """clip spect1"""
    result_clipRasterPath_spect1 = os.path.join(folderPath, "spect1")
    os.makedirs(result_clipRasterPath_spect1)
    clip_other_data(spect1_folder, frames_folderPath, result_clipRasterPath_spect1)
    
    """clip spect2"""
    result_clipRasterPath_spect2 = os.path.join(folderPath, "spect2")
    os.makedirs(result_clipRasterPath_spect2)
    clip_other_data(spect2_folder, frames_folderPath, result_clipRasterPath_spect2)
    
    """clip spect3"""
    result_clipRasterPath_spect3 = os.path.join(folderPath, "spect3")
    os.makedirs(result_clipRasterPath_spect3)
    clip_other_data(spect3_folder, frames_folderPath, result_clipRasterPath_spect3)
    
    """clip spect4"""
    result_clipRasterPath_spect4 = os.path.join(folderPath, "spect4")
    os.makedirs(result_clipRasterPath_spect4)
    clip_other_data(spect4_folder, frames_folderPath, result_clipRasterPath_spect4)
    
    """clip spect5"""
    result_clipRasterPath_spect5 = os.path.join(folderPath, "spect5")
    os.makedirs(result_clipRasterPath_spect5)
    clip_other_data(spect5_folder, frames_folderPath, result_clipRasterPath_spect5)
    
    """clip Clementine_Fe"""
    result_clipRasterPath_Clementine_Fe = os.path.join(folderPath, "Clementine_Fe")
    os.makedirs(result_clipRasterPath_Clementine_Fe)
    clip_other_data(clementine_Fe, frames_folderPath, result_clipRasterPath_Clementine_Fe)

    """clip Clementine_Ti"""
    result_clipRasterPath_Clementine_Ti = os.path.join(folderPath, "Clementine_Ti")
    os.makedirs(result_clipRasterPath_Clementine_Ti)
    clip_other_data(clementine_Ti, frames_folderPath, result_clipRasterPath_Clementine_Ti)
    
    """clip He3"""
    result_clipRasterPath_He3 = os.path.join(folderPath, "He3")
    os.makedirs(result_clipRasterPath_He3)
    clip_other_data(He3_folder, frames_folderPath, result_clipRasterPath_He3)
    
    """clip ling_Fe"""
    result_clipRasterPath_ling_Fe = os.path.join(folderPath, "ling_Fe")
    os.makedirs(result_clipRasterPath_ling_Fe)
    clip_other_data(ling_Fe_folder, frames_folderPath, result_clipRasterPath_ling_Fe)
    
    """clip ling_Ti"""
    result_clipRasterPath_ling_Ti = os.path.join(folderPath, "ling_Ti")
    os.makedirs(result_clipRasterPath_ling_Ti)
    clip_other_data(ling_Ti_folder, frames_folderPath, result_clipRasterPath_ling_Ti)
    
    """clip olivine"""
    result_clipRasterPath_olivine = os.path.join(folderPath, "olivine")
    os.makedirs(result_clipRasterPath_olivine)
    clip_other_data(olivine, frames_folderPath, result_clipRasterPath_olivine)
    
    """clip plagioclase"""
    result_clipRasterPath_plagioclase = os.path.join(folderPath, "plagioclase")
    os.makedirs(result_clipRasterPath_plagioclase)
    clip_other_data(plagioclase, frames_folderPath, result_clipRasterPath_plagioclase) 